In [3]:
# Imports 
from ppi_client.api.constants import ACCOUNTDATA_TYPE_ACCOUNT_NOTIFICATION, ACCOUNTDATA_TYPE_PUSH_NOTIFICATION, \
    ACCOUNTDATA_TYPE_ORDER_NOTIFICATION
from ppi_client.models.account_movements import AccountMovements
from ppi_client.models.bank_account_request import BankAccountRequest
from ppi_client.models.foreign_bank_account_request import ForeignBankAccountRequest, ForeignBankAccountRequestDTO
from ppi_client.models.cancel_bank_account_request import CancelBankAccountRequest
from ppi_client.models.order import Order
from ppi_client.ppi import PPI
from ppi_client.models.order_budget import OrderBudget
from ppi_client.models.order_confirm import OrderConfirm
from ppi_client.models.disclaimer import Disclaimer
from ppi_client.models.investing_profile import InvestingProfile
from ppi_client.models.investing_profile_answer import InvestingProfileAnswer
from ppi_client.models.instrument import Instrument
from datetime import datetime, timedelta
from ppi_client.models.estimate_bonds import EstimateBonds
import asyncio
import json
import traceback
import os

### Connection to Sandbox environment

In [14]:
#Change sandbox variable to False to connect to real account
sandbox_public_key='enZ1dE5HcnlHOWRiQlpCWXEyQXg='
sandbox_secret_key='Zjg2Zjc3NzItYTQ1Yy00ZWM3LTk3YzYtZjAwMzE0ODNlNTcy'

ppi = PPI(sandbox=True)

In [15]:
# Change login credential to connect to the API
ppi.account.login_api(sandbox_public_key,sandbox_secret_key)

{'creationDate': '2023-05-27T13:08:48-03:00',
 'expirationDate': '2023-05-27T14:38:48-03:00',
 'accessToken': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJQUEF1dGguQ2xhaW1zLkdlbmVyYWwuR3JhbnRUeXBlIjpbIkRBVE9TX1BFUlNPTkFMRVMiLCJFU0NSSVRVUkEiLCJMRUNUVVJBIl0sInJvbGUiOiJQUEF1dGguUm9sLkdlbmVyYWwuVXNlclRyYWRpbmciLCJQUEF1dGguQ2xhaW1zLkdlbmVyYWwuQ3VlbnRhcyI6IjE3MDY3IiwidW5pcXVlX25hbWUiOiJ1c3VhcmlvNDMzNTEiLCJuYW1laWQiOiI0MzM1MSIsIlBQQXV0aC5DbGFpbXMuR2VuZXJhbC5GaXNydE5hbWUiOiJOb21icmUgNDMzNTEiLCJQUEF1dGguQ2xhaW1zLkdlbmVyYWwuTGFzdE5hbWUiOiJBcGVsbGlkbyA0MzM1MSIsImVtYWlsIjoibWFpbDQzMzUxQHRlc3QuY29tIiwiUFBBdXRoLkNsYWltcy5HZW5lcmFsLkV4cGlyYXRpb25UaW1lIjoiMjAyMy8wNS8yNyAxNDozODo0OSIsIlBQQXV0aC5DbGFpbXMuR2VuZXJhbC5DbGllbnRJRCI6IkFQSV9DTElfUFlUSE9OIiwiaXNzIjoiUFBBdXRoLnNhbmRib3guUFBJIiwiYXVkIjoiaHR0cHM6Ly9hcGlfc2FuZGJveC5wb3J0Zm9saW9wZXJzb25hbC5jb20iLCJleHAiOjE2ODUyMDkxMjgsIm5iZiI6MTY4NTIwMzcyOH0.Br9wLGEEEPCCQ3mi0_9Mghi6jp0Va3rhMnlLTw-4Bio',
 'expires': 5399,
 'refreshToken': 'da6c49737e084af8af71426b6868f5

## Account Information

In [17]:
print("Getting accounts information")
account_numbers = ppi.account.get_accounts()
print(account_numbers)

Getting accounts information
[{'accountNumber': '151233', 'name': 'Cuenta 151233', 'officer': {'name': 'Portfolio Personal', 'eMail': 'consultas@portfoliopersonal.com', 'phone': '+5411 5252-7599'}, 'externalID': None}]


In [19]:
#Sandbox has only one account
for account in account_numbers:
    print(account)

{'accountNumber': '151233', 'name': 'Cuenta 151233', 'officer': {'name': 'Portfolio Personal', 'eMail': 'consultas@portfoliopersonal.com', 'phone': '+5411 5252-7599'}, 'externalID': None}


In [24]:
account_number = account_numbers[0]['accountNumber']
balances = ppi.account.get_available_balance(account_number)
#Print balances of the sandbox account
print(balances)

[{'name': 'Pesos', 'symbol': 'ARS', 'amount': -15461.57, 'settlement': 'INMEDIATA'}, {'name': 'Pesos', 'symbol': 'ARS', 'amount': -15461.57, 'settlement': 'A-24HS'}, {'name': 'Pesos', 'symbol': 'ARS', 'amount': -15461.57, 'settlement': 'A-48HS'}, {'name': 'Dolar Divisa | CCL', 'symbol': 'USD', 'amount': 0, 'settlement': 'INMEDIATA'}, {'name': 'Dolar Divisa | CCL', 'symbol': 'USD', 'amount': 0, 'settlement': 'A-24HS'}, {'name': 'Dolar Divisa | CCL', 'symbol': 'USD', 'amount': 0, 'settlement': 'A-48HS'}, {'name': 'Transferencia Dolar Divisa | CCL', 'symbol': 'USD', 'amount': 14.43, 'settlement': 'INMEDIATA'}, {'name': 'Dolar Billete | MEP', 'symbol': 'USD', 'amount': 0, 'settlement': 'INMEDIATA'}, {'name': 'Dolar Billete | MEP', 'symbol': 'USD', 'amount': 0, 'settlement': 'A-24HS'}, {'name': 'Dolar Billete | MEP', 'symbol': 'USD', 'amount': 0, 'settlement': 'A-48HS'}, {'name': 'Transferencia Dolar Billete | MEP', 'symbol': 'USD', 'amount': 17.34, 'settlement': 'INMEDIATA'}]


In [27]:
# Getting balance and positions
balances_positions = ppi.account.get_balance_and_positions(account_number)
print(balances_positions)

{'groupedAvailability': [{'currency': 'Pesos', 'availability': [{'name': 'Pesos', 'symbol': 'ARS', 'amount': 2206.43, 'settlement': 'INMEDIATA'}, {'name': 'Pesos', 'symbol': 'ARS', 'amount': 0, 'settlement': 'A-24HS'}, {'name': 'Pesos', 'symbol': 'ARS', 'amount': 0, 'settlement': 'A-48HS'}]}, {'currency': 'Dolar Divisa | CCL', 'availability': [{'name': 'Dolar Divisa | CCL', 'symbol': 'USD', 'amount': 14.43, 'settlement': 'INMEDIATA'}, {'name': 'Dolar Divisa | CCL', 'symbol': 'USD', 'amount': 0, 'settlement': 'A-24HS'}, {'name': 'Dolar Divisa | CCL', 'symbol': 'USD', 'amount': 0, 'settlement': 'A-48HS'}]}, {'currency': 'Dolar Billete | MEP', 'availability': [{'name': 'Dolar Billete | MEP', 'symbol': 'USD', 'amount': 2.91, 'settlement': 'INMEDIATA'}, {'name': 'Dolar Billete | MEP', 'symbol': 'USD', 'amount': 0, 'settlement': 'A-24HS'}, {'name': 'Dolar Billete | MEP', 'symbol': 'USD', 'amount': 0, 'settlement': 'A-48HS'}]}], 'groupedInstruments': [{'name': 'ACCIONES', 'instruments': [{'ti

In [28]:
#Positions in the sandbox account
for instruments in balances_positions["groupedInstruments"]:
    print("Instrument %s " % instruments['name'])
    for instrument in instruments['instruments']:
        print("Ticker %s Price %s Amount %s" % (
            instrument['ticker'], instrument['price'], instrument['amount']))

Instrument ACCIONES 
Ticker AGRO Price 70.6 Amount 35300.0
Ticker ALUA Price 240.25 Amount 55978.25
Ticker GGAL Price 447.85 Amount 317973.5
Ticker SUPV Price 191.3 Amount 19130.0
Instrument BONOS 
Ticker T2V1 Price 100.49 Amount 33664.15
Instrument CEDEARS 
Ticker ABEV Price 3431 Amount 89206
Ticker AZN Price 14528.5 Amount 14528.5
Ticker BABA Price 4557.5 Amount 22787.5
Ticker RIO Price 6784 Amount 33920
Ticker VIST Price 8579.5 Amount 102954
Ticker WFC Price 3125 Amount 31250
Instrument FCI 
Ticker TT.AHORRO.A Price 11.2893 Amount 142438.59


## Market Data

In [41]:
# Search Historic MarketData via Datetime python module on a daily frame
print("\nSearching MarketData")
market_data = ppi.marketdata.search("GGAL", "Acciones", "A-48HS", datetime(2023, 3, 1), datetime.now())
for ins in market_data:
    print("%s - %s - Volume %s - Opening %s - Min %s - Max %s" % (
        ins['date'], ins['price'], ins['volume'], ins['openingPrice'], ins['min'], ins['max']))


Searching MarketData
2023-03-01T00:00:00-03:00 - 492.4 - Volume 359113792 - Opening 488.55 - Min 480.35 - Max 500
2023-03-02T00:00:00-03:00 - 467.1 - Volume 707777728 - Opening 490 - Min 466.5 - Max 493.5
2023-03-03T00:00:00-03:00 - 478.2 - Volume 599348352 - Opening 467 - Min 465 - Max 482.35
2023-03-06T00:00:00-03:00 - 488 - Volume 579470592 - Opening 477 - Min 465 - Max 490.5
2023-03-07T00:00:00-03:00 - 475.45 - Volume 432177568 - Opening 493 - Min 473 - Max 496.4
2023-03-08T00:00:00-03:00 - 497.25 - Volume 901319808 - Opening 479.5 - Min 467.5 - Max 500
2023-03-09T00:00:00-03:00 - 486.2 - Volume 568237184 - Opening 498.5 - Min 485.7 - Max 506.9
2023-03-10T00:00:00-03:00 - 464.15 - Volume 944450624 - Opening 486.25 - Min 457 - Max 489.5
2023-03-13T00:00:00-03:00 - 445.05 - Volume 972578816 - Opening 460 - Min 444 - Max 466.9
2023-03-14T00:00:00-03:00 - 431.85 - Volume 845406208 - Opening 461 - Min 430.05 - Max 463.5
2023-03-15T00:00:00-03:00 - 408.3 - Volume 868635584 - Opening 420

In [42]:
# Search Intraday MarketData
print("\nSearching Intraday MarketData")
intraday_market_data = ppi.marketdata.intraday("GGAL", "Acciones", "A-48HS")
#Doesn't work if market is closed
for intra in intraday_market_data:
    print(intra)


Searching Intraday MarketData


In [33]:
# Search Current Order Book
print("\nSearching Current Book")
current_book = ppi.marketdata.book("GGAL", "Acciones", "A-48HS")
#If market is closed it'll throw no offers or bids
print(current_book)


Searching Current Book
{'date': '0001-01-01T00:00:00-03:00', 'offers': [], 'bids': []}


## Order management

In [43]:
# Get budget
print("\nGet budget for the order")
budget_order = ppi.orders.budget(OrderBudget(account_number, 10000, 150, "GGAL", "ACCIONES", "Dinero",
                                             "PRECIO-LIMITE", "HASTA-SU-EJECUCIÓN", None, "Compra",
                                             "INMEDIATA"))
print(budget_order)
disclaimers_order = budget_order['disclaimers']



Get budget for the order


Exception: Saldo insuficiente para la operación solicitada. Se requiere de AR$9.980,78 y usted dispone de AR$-15.461,57

In [44]:

# Confirm order
print("\nConfirm order")
accepted_disclaimers = []
for disclaimer in disclaimers_order:
    accepted_disclaimers.append(Disclaimer(disclaimer['code'], True))
confirmation = ppi.orders.confirm(OrderConfirm(account_number, 10000, 150, "GGAL", "ACCIONES", "Dinero",
                                               "PRECIO-LIMITE", "HASTA-SU-EJECUCIÓN", None, "Compra"
                                               , "INMEDIATA", accepted_disclaimers, None))
print(confirmation)
order_id = confirmation["id"]




Confirm order


NameError: name 'disclaimers_order' is not defined

## Bond analysis

In [46]:
# Estimate bond
print("\nEstimate bond")
estimate = ppi.marketdata.estimate_bonds(EstimateBonds(ticker="CUAP", date=datetime(2023,4,4), 
quantityType="PAPELES", quantity=100, price=4555))
print(estimate)



Estimate bond
{'flows': [{'cuttingDate': '2023-06-30T00:00:00-03:00', 'residualValue': 1, 'rent': 130.307576585739, 'amortization': 0, 'total': 130.307576585739, 'quantity': 0, 'ticker': None, 'currency': None, 'name': None}, {'cuttingDate': '2023-12-31T00:00:00-03:00', 'residualValue': 1, 'rent': 130.307576585739, 'amortization': 0, 'total': 130.307576585739, 'quantity': 0, 'ticker': None, 'currency': None, 'name': None}, {'cuttingDate': '2024-06-30T00:00:00-03:00', 'residualValue': 1, 'rent': 130.307576585739, 'amortization': 0, 'total': 130.307576585739, 'quantity': 0, 'ticker': None, 'currency': None, 'name': None}, {'cuttingDate': '2024-12-31T00:00:00-03:00', 'residualValue': 1, 'rent': 130.307576585739, 'amortization': 0, 'total': 130.307576585739, 'quantity': 0, 'ticker': None, 'currency': None, 'name': None}, {'cuttingDate': '2025-06-30T00:00:00-03:00', 'residualValue': 1, 'rent': 130.307576585739, 'amortization': 0, 'total': 130.307576585739, 'quantity': 0, 'ticker': None, 'c